In [1]:
import Levenshtein

In [5]:
# Chuỗi phoneme gốc và chuỗi do người dùng phát âm
correct_phoneme = "dʒænjuɛɹi"
test_phoneme = "ɛnudɹi"

# Lấy danh sách các thao tác cần thiết
opcodes = Levenshtein.opcodes(correct_phoneme, test_phoneme)

print(f"So sánh '{correct_phoneme}' và '{test_phoneme}':")
print(f"Các thao tác cần thực hiện: {opcodes}\n")

# Diễn giải kết quả cho dễ hiểu
print("Diễn giải chi tiết:")
for tag, i1, i2, j1, j2 in opcodes:
    if tag == 'equal':
        # Bỏ qua vì chúng giống nhau
        print(f"- Giống nhau: '{correct_phoneme[i1:i2]}'")
    elif tag == 'replace':
        print(f"- THAY THẾ: Ký tự '{correct_phoneme[i1:i2]}' (tại vị trí {i1}) bằng '{test_phoneme[j1:j2]}'")
    elif tag == 'delete':
        print(f"- XÓA: Ký tự '{correct_phoneme[i1:i2]}' (tại vị trí {i1})")
    elif tag == 'insert':
        print(f"- CHÈN: Ký tự '{test_phoneme[j1:j2]}' (tại vị trí {i1})")

So sánh 'dʒænjuɛɹi' và 'ɛnudɹi':
Các thao tác cần thực hiện: [('replace', 0, 1, 0, 1), ('delete', 1, 3, 1, 1), ('equal', 3, 4, 1, 2), ('delete', 4, 5, 2, 2), ('equal', 5, 6, 2, 3), ('replace', 6, 7, 3, 4), ('equal', 7, 9, 4, 6)]

Diễn giải chi tiết:
- THAY THẾ: Ký tự 'd' (tại vị trí 0) bằng 'ɛ'
- XÓA: Ký tự 'ʒæ' (tại vị trí 1)
- Giống nhau: 'n'
- XÓA: Ký tự 'j' (tại vị trí 4)
- Giống nhau: 'u'
- THAY THẾ: Ký tự 'ɛ' (tại vị trí 6) bằng 'd'
- Giống nhau: 'ɹi'


In [4]:
import Levenshtein

# Chuỗi phoneme gốc và chuỗi do người dùng phát âm
correct_phoneme = "dʒænjuɛɹi"
test_phoneme = "tʃæɹuɝi"

# Lấy danh sách các thao tác cần thiết
opcodes = Levenshtein.opcodes(correct_phoneme, test_phoneme)

print(f"So sánh '{correct_phoneme}' và '{test_phoneme}':")
print(f"Các thao tác cần thực hiện: {opcodes}\n")

# ANSI color codes
GREEN = '\033[92m'  # Màu xanh lá
RED = '\033[91m'    # Màu đỏ
RESET = '\033[0m'   # Reset màu

# Tạo chuỗi có màu
colored_output = ""

print("Diễn giải chi tiết:")
for tag, i1, i2, j1, j2 in opcodes:
    if tag == 'equal':
        # Giống nhau -> màu xanh
        colored_output += GREEN + correct_phoneme[i1:i2] + RESET
        print(f"- Giống nhau: '{correct_phoneme[i1:i2]}'")
    elif tag == 'replace':
        # Thay thế -> màu đỏ
        colored_output += RED + correct_phoneme[i1:i2] + RESET
        print(f"- THAY THẾ: Ký tự '{correct_phoneme[i1:i2]}' (tại vị trí {i1}) bằng '{test_phoneme[j1:j2]}'")
    elif tag == 'delete':
        # Xóa -> màu đỏ
        colored_output += RED + correct_phoneme[i1:i2] + RESET
        print(f"- XÓA: Ký tự '{correct_phoneme[i1:i2]}' (tại vị trí {i1})")
    elif tag == 'insert':
        # Chèn -> không hiển thị gì trong correct_phoneme
        print(f"- CHÈN: Ký tự '{test_phoneme[j1:j2]}' (tại vị trí {i1})")

print(f"\nKết quả hiển thị correct_phoneme với màu:")
print(colored_output)
print(test_phoneme)
print(f"\nChú thích: {GREEN}Xanh = đúng{RESET}, {RED}Đỏ = sai (thay thế/xóa){RESET}")

So sánh 'dʒænjuɛɹi' và 'tʃæɹuɝi':
Các thao tác cần thực hiện: [('replace', 0, 2, 0, 2), ('equal', 2, 3, 2, 3), ('replace', 3, 4, 3, 4), ('delete', 4, 5, 4, 4), ('equal', 5, 6, 4, 5), ('replace', 6, 7, 5, 6), ('delete', 7, 8, 6, 6), ('equal', 8, 9, 6, 7)]

Diễn giải chi tiết:
- THAY THẾ: Ký tự 'dʒ' (tại vị trí 0) bằng 'tʃ'
- Giống nhau: 'æ'
- THAY THẾ: Ký tự 'n' (tại vị trí 3) bằng 'ɹ'
- XÓA: Ký tự 'j' (tại vị trí 4)
- Giống nhau: 'u'
- THAY THẾ: Ký tự 'ɛ' (tại vị trí 6) bằng 'ɝ'
- XÓA: Ký tự 'ɹ' (tại vị trí 7)
- Giống nhau: 'i'

Kết quả hiển thị correct_phoneme với màu:
dʒænjuɛɹi
tʃæɹuɝi

Chú thích: Xanh = đúng, Đỏ = sai (thay thế/xóa)


In [ ]:
import Levenshtein
import json

def compare_phonemes(correct_phoneme, test_phoneme):
    """
    So sánh 2 chuỗi phoneme và trả về kết quả dạng JSON cho FE
    """
    # Lấy danh sách các thao tác cần thiết
    opcodes = Levenshtein.opcodes(correct_phoneme, test_phoneme)
    
    # Tạo danh sách kết quả cho từng ký tự
    result = []
    
    for tag, i1, i2, j1, j2 in opcodes:
        if tag == 'equal':
            # Giống nhau -> màu xanh
            for i in range(i1, i2):
                result.append({
                    "char": correct_phoneme[i],
                    "position": i,
                    "status": "correct",
                    "color": "green"
                })
        elif tag == 'replace':
            # Thay thế -> màu đỏ
            for i in range(i1, i2):
                result.append({
                    "char": correct_phoneme[i],
                    "position": i,
                    "status": "wrong",
                    "color": "red",
                    "replaced_with": test_phoneme[j1 + (i - i1)] if j1 + (i - i1) < j2 else None
                })
        elif tag == 'delete':
            # Xóa -> màu đỏ
            for i in range(i1, i2):
                result.append({
                    "char": correct_phoneme[i],
                    "position": i,
                    "status": "delete",
                    "color": "red"
                })
        elif tag == 'insert':
            # Chèn -> không hiển thị trong correct_phoneme nhưng ghi nhận
            pass
    
    # Tính điểm chính xác
    total_chars = len(correct_phoneme)
    correct_chars = sum(1 for item in result if item["status"] == "correct")
    accuracy = round((correct_chars / total_chars * 100), 2) if total_chars > 0 else 0
    
    return {
        "correct_phoneme": correct_phoneme,
        "test_phoneme": test_phoneme,
        "accuracy": accuracy,
        "details": result,
        "summary": {
            "total": total_chars,
            "correct": correct_chars,
            "errors": total_chars - correct_chars
        }
    }


# Test code
if __name__ == "__main__":
    correct_phoneme = "dʒænjuɛɹi"
    test_phoneme = "tʃæɹuɝi"
    
    result = compare_phonemes(correct_phoneme, test_phoneme)
    
    # In kết quả dạng JSON đẹp
    print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "correct_phoneme": "dʒænjuɛɹi",
  "test_phoneme": "tʃæɹuɝi",
  "accuracy": 33.33,
  "details": [
    {
      "char": "d",
      "position": 0,
      "status": "replace",
      "color": "red",
      "replaced_with": "t"
    },
    {
      "char": "ʒ",
      "position": 1,
      "status": "replace",
      "color": "red",
      "replaced_with": "ʃ"
    },
    {
      "char": "æ",
      "position": 2,
      "status": "correct",
      "color": "green"
    },
    {
      "char": "n",
      "position": 3,
      "status": "replace",
      "color": "red",
      "replaced_with": "ɹ"
    },
    {
      "char": "j",
      "position": 4,
      "status": "delete",
      "color": "red"
    },
    {
      "char": "u",
      "position": 5,
      "status": "correct",
      "color": "green"
    },
    {
      "char": "ɛ",
      "position": 6,
      "status": "replace",
      "color": "red",
      "replaced_with": "ɝ"
    },
    {
      "char": "ɹ",
      "position": 7,
      "status": "delete",
     